In [22]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import pandas_ta as ta  # Alternative if TA-Lib is not installed

# Load your OHLC data
# Ensure your data has columns: 'Date', 'Open', 'High', 'Low', 'Close'
# Replace 'your_data.csv' with your actual data file path
ohlc = pd.read_csv('/workspaces/Futures-First/BackTest/data/ZL Mar25-May25 Calendar_1min.csv', parse_dates=['Date'], index_col='Date')

# Define a function to calculate the ATR trailing stops
def calculate_atr_trailing_stop(df, period=14, multiplier=1.5):
    atr = df.ta.atr(length=period)
    df['ATR'] = atr
    df['Trailing_Stop_Long'] = df['Close'] - (multiplier * df['ATR'])
    df['Trailing_Stop_Short'] = df['Close'] + (multiplier * df['ATR'])
    return df

# Resample data and calculate ATR trailing stops for each time frame
time_frames = ['1min', '10min', '30min', '45min', '1H', '1D']
multiplier = 1.5  # Adjust as needed

for time_frame in time_frames:
    resampled_data = ohlc.resample(time_frame).agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last'
    }).dropna()

    # Calculate ATR trailing stops
    resampled_data = calculate_atr_trailing_stop(resampled_data, period=14, multiplier=multiplier)

    # Create a Plotly figure for each time frame
    fig = go.Figure()

    # Add OHLC candlestick chart
    fig.add_trace(go.Candlestick(
        x=resampled_data.index,
        open=resampled_data['Open'],
        high=resampled_data['High'],
        low=resampled_data['Low'],
        close=resampled_data['Close'],
        name='OHLC'
    ))

    # Add ATR trailing stop lines
    fig.add_trace(go.Scatter(
        x=resampled_data.index,
        y=resampled_data['Trailing_Stop_Long'],
        mode='lines',
        line=dict(color='green', width=2),
        name='Trailing Stop Long'
    ))
    fig.add_trace(go.Scatter(
        x=resampled_data.index,
        y=resampled_data['Trailing_Stop_Short'],
        mode='lines',
        line=dict(color='red', width=2),
        name='Trailing Stop Short'
    ))

    # Update layout
    fig.update_layout(
        title=f'ATR Trailing Stops for {time_frame} Time Frame',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        template='plotly_dark'
    )

    # Show the figure
    fig.show()


ImportError: cannot import name 'NaN' from 'numpy' (/home/codespace/.local/lib/python3.12/site-packages/numpy/__init__.py)